In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("./data.csv")
newData = pd.read_csv("./recipes_add.csv")

In [17]:
x = df["ID"].max()

In [16]:
### Select rows from recipe_add (you can specify your condition here)
selected_rows = newData.loc[[0, 1]]
selected_rows

,Title,Prep_Time,Cook_Time,Cuisine,Course,Diet,Cleaned_Ingredients,Image
0,rrttv d,30,30,Gujarati Recipes﻿,Lunch,High Protein Non Vegetarian,"['allpurpose flour', 'amaranth']",NaN
1,sdf,30,30,Gujarati Recipes﻿,Dessert,Non Vegeterian,['aluminum foil sifter foilcovered cardboard c...,NaN


In [18]:
selected_rows["ID"] = range(x + 1, x + 1 + len(selected_rows))

In [24]:
updated_data_df = pd.concat([df, selected_rows], ignore_index=True)

## Output
### Save the updated DataFrame to data.csv
updated_data_df.to_csv("data.csv", index=False)

In [23]:
from transformers import AutoModel
import torch

model = AutoModel.from_pretrained("./jina_clip_v1_model", trust_remote_code=True)
model = torch.load("./jina.pt")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

2024-08-27 20:59:46.135932: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 20:59:46.303971: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 20:59:46.353169: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 20:59:46.683463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 20:59:48.760854: W tensorflow/compiler/tf2

JinaCLIPModel(
  (text_model): HFTextEncoder(
    (transformer): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30528, 768, padding_idx=0)
        (token_type_embeddings): Embedding(2, 768)
      )
      (emb_drop): Dropout(p=0.1, inplace=False)
      (emb_ln): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (encoder): BertEncoder(
        (layers): ModuleList(
          (0-11): 12 x Block(
            (mixer): MHA(
              (Wqkv): LinearResidual(in_features=768, out_features=2304, bias=True)
              (inner_attn): SelfAttention(
                (drop): Dropout(p=0.1, inplace=False)
              )
              (inner_cross_attn): CrossAttention(
                (drop): Dropout(p=0.1, inplace=False)
              )
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (dropout1): Dropout(p=0.1, inplace=False)
            (drop_path1): StochasticDepth(p=0.0, mode=row)
        

In [25]:
## Input
### Import necessary libraries
import json
import numpy as np


### Load existing embeddings JSON file
json_file_path = "embeddings.json"
try:
    with open(json_file_path, "r") as json_file:
        embeddings_dict = json.load(json_file)
except FileNotFoundError:
    # If the file does not exist, start with an empty dictionary
    embeddings_dict = {}

In [26]:
embeddings_dict.keys()

dict_keys(['4529', '4640', '5978', '7092', '6684', '2096', '2267', '4406', '4399', '1798', '6347', '6825', '5638', '7680', '1970', '1971', '5636', '4112', '3983', '5170', '5825', '7444', '2942', '3915', '7015', '2010', '4150', '2934', '1724', '2670', '2641', '7525', '3739', '1322', '2271', '1249', '7341', '3055', '4146', '3906', '2827', '6367', '2905', '4999', '7009', '5731', '2745', '1500', '5776', '4170', '7072', '4574', '7589', '6423', '3135', '5805', '4771', '3575', '3672', '5612', '7089', '3456', '6944', '3208', '6621', '6058', '4762', '2086', '7150', '2059', '3297', '6031', '3893', '3960', '5094', '3275', '5375', '6537', '2594', '4219', '1978', '2017', '6889', '1804', '4178', '6725', '1843', '5296', '5200', '1422', '6149', '5453', '4279', '1584', '5454', '7602', '7790', '1557', '1301', '5403', '5175', '2123', '7673', '6531', '6236', '1367', '4114', '1102', '6625', '6023', '7791', '1495', '5984', '3844', '6003', '3935', '1146', '3530', '4280', '3666', '4924', '6229', '5613', '4523

In [35]:
def compute_average_embedding(title_text=None, image=None):
    embeddings = []
    if title_text:
        title_embedding = torch.tensor(model.encode_text(title_text)).to(device)
        embeddings.append(title_embedding)
    if image is not None:
        try:
            # Check if `image` is a file path or a file-like object
            if isinstance(image, str) or hasattr(image, "read"):
                # Open the image file
                image = Image.open(image)
            # image = Image.open(image)
            image_embedding = torch.tensor(model.encode_image(image)).to(device)
            embeddings.append(image_embedding)
        except:
            pass
    if len(embeddings) == 0:
        return list(torch.zeros(768).cpu().numpy())
    avg_embedding = torch.mean(torch.stack(embeddings), dim=0)
    return list(avg_embedding.cpu().numpy())

In [39]:
## Model
### Iterate over the rows of selected_rows
for _, row in selected_rows.iterrows():
    print(row["Image"])

    combined_embedding = compute_average_embedding(row["Title"], row["Image"])
    # Combine embeddings (you can use the average or any other method)
    combined_embedding = [float(x) for x in combined_embedding]
    # Store in dictionary with ID as the key
    embeddings_dict[str(row["ID"])] = (
        combined_embedding  # Convert to list for JSON serialization
    )

nan
nan


In [40]:
## Output
### Save the updated embeddings dictionary back to the JSON file
with open(json_file_path, "w") as json_file:
    json.dump(embeddings_dict, json_file, indent=4)

In [41]:
## Input
### Import necessary libraries
import json
import numpy as np


### Load existing embeddings JSON file
json_file_path = "embeddings.json"
try:
    with open(json_file_path, "r") as json_file:
        embeddings_dict = json.load(json_file)
except FileNotFoundError:
    # If the file does not exist, start with an empty dictionary
    embeddings_dict = {}

In [48]:
torch.tensor(list(embeddings_dict.values())).shape

torch.Size([5046, 768])